In [ ]:
import pandas as pd
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing as mp
from multiprocessing import cpu_count
from sklearn.model_selection import StratifiedGroupKFold, KFold

import tensorflow as tf
cpu_count()

In [2]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Bidirectional, LSTM, Dense, Lambda, Activation, BatchNormalization, Dropout
from keras.optimizers import Adam

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from skimage.transform import resize
import json
from tqdm import tqdm
import os

In [4]:
train_df_leg = pd.read_csv('/dataset/input/asl-signs/train.csv')

In [5]:
train_df = pd.read_csv('/dataset/input/asl-fingerspelling/train.csv')

In [6]:
train_df

,path,file_id,sequence_id,participant_id,phrase
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road
...,...,...,...,...,...
67203,train_landmarks/2118949241.parquet,2118949241,388192924,88,431-366-2913
67204,train_landmarks/2118949241.parquet,2118949241,388225542,154,994-392-3850
67205,train_landmarks/2118949241.parquet,2118949241,388232076,95,https://www.tianjiagenomes.com
67206,train_landmarks/2118949241.parquet,2118949241,388235284,36,90 kerwood circle


In [7]:
train_df.sequence_id.nunique()

67208

In [8]:
train_df.head()

,path,file_id,sequence_id,participant_id,phrase
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road


In [9]:
pd.set_option('display.max_columns', 500)

In [10]:
# tmp = pd.read_parquet('/dataset/input/asl-fingerspelling/train_landmarks/5414471.parquet')

In [11]:
# tmp.head(127)

In [12]:
# del(tmp)

In [13]:
import re
import gc
def transform_data(tmp):
#     res = pd.DataFrame(columns = ['frame', 'type', 'landmark_index', 'x', 'y', 'z'])
    res = pd.DataFrame(columns = ['x', 'y', 'z'])
    for i, row in tqdm(tmp.iterrows()):
        tmp_frame_num = int(row.frame)
        tmp_idx = np.array(tmp.columns[1:544].copy())
        tmp_idx = np.array(list(map(lambda v: re.sub('x_','', v) ,tmp_idx)))
#         tmp_frame_num = tmp.iloc[0, 0]
#         tmp_cords = tmp.iloc[0, 1:]
        tmp_cords = row[1:]
        tmp_cords_values = tmp_cords.values.flatten().tolist()
        tmp_reshaped = pd.DataFrame(np.reshape(tmp_cords_values, (3, 543))).T.rename(columns = {0:'x', 1:'y', 2:'z'})
#         tmp_reshaped['type'] = np.array(list(map(lambda v: re.sub(r'_\d+','', v) ,tmp_idx)))
#         tmp_reshaped['landmark_index'] = np.array(list(map(lambda v: re.sub(r'.*_','', v) ,tmp_idx)))
#         tmp_reshaped['frame'] = tmp_frame_num
#         cols = tmp_reshaped.columns.tolist()
#         cols = ['frame', 'type', 'landmark_index', 'x', 'y', 'z']

        res = pd.concat([res, tmp_reshaped])
        del tmp_idx
        del tmp_cords
        del tmp_cords_values
        del tmp_reshaped
    del tmp    
    gc.collect()
    return res 


In [14]:
# data = tmp.loc[tmp.index == 1816796431]
# data = transform_data(data)

In [15]:
# tmp_idx

In [16]:
# tmp_reshaped

In [17]:
pd.read_parquet('/dataset/input/asl-signs/train_landmark_files/16069/100015657.parquet')

,frame,row_id,type,landmark_index,x,y,z
0,103,103-face-0,face,0,0.437886,0.437599,-0.051134
1,103,103-face-1,face,1,0.443258,0.392901,-0.067054
2,103,103-face-2,face,2,0.443997,0.409998,-0.042990
3,103,103-face-3,face,3,0.435256,0.362771,-0.039492
4,103,103-face-4,face,4,0.443780,0.381762,-0.068013
...,...,...,...,...,...,...,...
57010,207,207-right_hand-16,right_hand,16,NaN,NaN,NaN
57011,207,207-right_hand-17,right_hand,17,NaN,NaN,NaN
57012,207,207-right_hand-18,right_hand,18,NaN,NaN,NaN
57013,207,207-right_hand-19,right_hand,19,NaN,NaN,NaN


In [18]:
# pd.read_parquet('/dataset/input/asl-fingerspelling/train_landmarks/1019715464.parquet')[:543].type.value_counts()

In [19]:
# pd.read_parquet('/dataset/input/asl-fingerspelling/train_landmarks/1019715464.parquet')[543:543*2].type.value_counts()

In [20]:
# pd.read_parquet('/dataset/input/asl-fingerspelling/supplemental_landmarks/1032110484.parquet')

In [21]:
# LIP = [
#     61, 185, 40, 39, 37, 267, 269, 270, 409,
#     291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
#     78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
#     95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
# ]

# FACE = [f'x_face_{i}' for i in LIP] + [f'y_face_{i}' for i in LIP] + [f'z_face_{i}' for i in LIP]
# LHAND = [f'x_left_hand_{i}' for i in range(21)] + [f'y_left_hand_{i}' for i in range(21)] + [f'z_left_hand_{i}' for i in range(21)]
# RHAND = [f'x_right_hand_{i}' for i in range(21)] + [f'y_right_hand_{i}' for i in range(21)] + [f'z_right_hand_{i}' for i in range(21)]
# POSE = [f'x_pose_{i}' for i in range(33)] + [f'y_pose_{i}' for i in range(33)] + [f'z_pose_{i}' for i in range(33)]

# SEL_COLS = FACE + LHAND + RHAND + POSE
# FRAME_LEN = 128

In [22]:
# def load_relevant_data_subset(pq_path):
#     return pd.read_parquet(pq_path, columns=SEL_COLS)

# for file_id in tqdm(df.file_id.unique()):
#     pqfile = f"{inpdir}/train_landmarks/{file_id}.parquet"
#     if not os.path.isdir("tfds"): os.mkdir("tfds")
#     tffile = f"tfds/{file_id}.tfrecord"
#     seq_refs = df.loc[df.file_id == file_id]
#     seqs = load_relevant_data_subset(pqfile)
    
#     with tf.io.TFRecordWriter(tffile) as file_writer:
#         for seq_id, phrase in zip(seq_refs.sequence_id, seq_refs.phrase_bytes):
#             frames = seqs.iloc[seqs.index == seq_id]
#             frames128 = frames.fillna(-10).to_numpy()
#             frames128 = resize(frames128, (FRAME_LEN, len(SEL_COLS)))
#             frames = pd.DataFrame(data = frames128, columns=frames.columns)
            
#             features = {COL: tf.train.Feature(float_list=tf.train.FloatList(value=frames[COL])) for COL in SEL_COLS}
#             features["phrase"] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[phrase]))
#             record_bytes = tf.train.Example(features=tf.train.Features(feature=features)).SerializeToString()
#             file_writer.write(record_bytes)

In [23]:
import numpy as np
import gc
ROWS_PER_FRAME = 543
def load_relevant_data_subset(pq_path, sequence_id):
    data_columns = ['x', 'y', 'z']
#     data = pd.read_parquet(pq_path, columns=data_columns)
    
    tmp = pd.read_parquet(pq_path)
    data = tmp.loc[tmp.index == sequence_id]
    data = transform_data(data)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    del tmp
    gc.collect()
    return data.astype(np.float32)

In [24]:
# import json
# with open('/dataset/input/asl-signs/sign_to_prediction_index_map.json') as json_file:
#     LABEL_DICT = json.load(json_file)

In [25]:
import json
with open('/dataset/input/asl-fingerspelling/character_to_prediction_index.json') as json_file:
    CHAR_DICT = json.load(json_file)

In [26]:
# alphabets = "abcdefghijklmnopqrstuvwxyz1234567890 -+&/_"
max_str_len = 64 # max length of input labels changed from 24
num_of_characters = len(CHAR_DICT) + 1 # +1 for ctc pseudo blank
num_of_timestamps = 64 # max length of predicted labels


def label_to_num(label):
    label_num = []
    for ch in label:
        label_num.append(CHAR_DICT[ch])
        
    return np.array(label_num)

# def num_to_label(num):
#     ret = ""
#     for ch in num:
#         if ch == -1:  # CTC Blank
#             break
#         else:
#             ret+=alphabets[ch]
#     return ret

In [27]:
train_size = len(train_df)
train_y = np.ones([train_size, max_str_len]) * -1
train_label_len = np.zeros([train_size, 1])
train_input_len = np.ones([train_size, 1]) * (num_of_timestamps-2)
train_output = np.zeros([train_size])

LAB_DICT = {}
for i in range(len(train_df)):
    LAB_DICT[train_df.loc[i, 'phrase']] = np.ones(max_str_len) * -1
#     train_label_len[i] = len(train.loc[i, 'sign'])
#     train_y[i, 0:len(train.loc[i, 'sign'])]= label_to_num(train.loc[i, 'sign'])
    LAB_DICT[train_df.loc[i, 'phrase']][0:len(train_df.loc[i, 'phrase'])] = label_to_num(train_df.loc[i, 'phrase'])
    LAB_DICT[train_df.loc[i, 'phrase']] = LAB_DICT[train_df.loc[i, 'phrase']].astype('int64')

In [ ]:
LAB_DICT

In [29]:
def _float_array_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [30]:
def encode_row(row):
    sequence_id = int(row.sequence_id)
    participant_id = int(row.participant_id)
    phrase = LAB_DICT[row.phrase]    
#     coordinates = load_relevant_data_subset(f'/dataset/input/asl-fingerspelling/train_landmarks/{row.path}')
    coordinates = load_relevant_data_subset(f'/dataset/input/asl-fingerspelling/{row.path}', sequence_id)

#     coordinates = coordinates.loc[train_df.sequence_id == sequence_id]
    coordinates_encoded = coordinates.tobytes()
    
    record_bytes = tf.train.Example(features=tf.train.Features(feature={
                'coordinates': tf.train.Feature(bytes_list=tf.train.BytesList(value=[coordinates_encoded])),
                'participant_id': tf.train.Feature(int64_list=tf.train.Int64List(value=[participant_id])),
                'sequence_id':tf.train.Feature(int64_list=tf.train.Int64List(value=[sequence_id])),
                'phrase':tf.train.Feature(int64_list=tf.train.Int64List(value=phrase)),
                })).SerializeToString()
    
    del sequence_id
    del participant_id
    del phrase
    del coordinates
    del coordinates_encoded
    gc.collect()
    return record_bytes

def process_chunk(chunk, tfrecord_name):
    options = tf.io.TFRecordOptions(compression_type='GZIP', compression_level=9)
    with tf.io.TFRecordWriter(tfrecord_name, options=options) as file_writer:
        for i, row in tqdm(chunk.iterrows()):
            record_bytes = encode_row(row)
            file_writer.write(record_bytes)
            del record_bytes
            gc.collect()
        file_writer.close()

In [31]:
row = train_df.iloc[0]
coordinates = load_relevant_data_subset(f'/dataset/input/asl-fingerspelling/{row.path}', row.sequence_id)
coordinates_encoded = coordinates.tobytes()
participant_id = int(row.participant_id)
sequence_id = int(row.sequence_id)
# sign = int(LABEL_DICT[row.sign])
phrase = LAB_DICT[row.phrase].tobytes()
# phrase_encoded = phrase.tobytes()

123it [00:00, 248.18it/s]


In [32]:
# sign

In [33]:
record_bytes = tf.train.Example(features=tf.train.Features(feature={
            'coordinates': tf.train.Feature(bytes_list=tf.train.BytesList(value=[coordinates_encoded])),
            'participant_id': tf.train.Feature(int64_list=tf.train.Int64List(value=[participant_id])),
            'sequence_id':tf.train.Feature(int64_list=tf.train.Int64List(value=[sequence_id])),
            'phrase':tf.train.Feature(bytes_list=tf.train.BytesList(value=[phrase])), 
            }))

In [34]:
# record_bytes

features {
  feature {
    key: "coordinates"
    value {
      bytes_list {
        value: "\037\3515?\255\000\007?N\031{\275\366/3?\230\237\372>0i\265\275\350\2454?\301\253\000?\021\376Y\275\271\0271??\242\354>\235\'`\275\211\0353?]\235\365>\331\377\272\275\365\2643?\274\372\357>zO\246\275q\2605?\006N\343>J\236\336\274\200C\034?\242%\347>\262\037f=\344p6?\2624\330>\204%\024\274v\2076?\211\260\321>\232\003\365\273\223w7?\221W\274>*]\336<\252\3735?v\017\010?r\232w\275\272\0366?\360\r\t?\267\233e\275\370S6?\310\257\t?\017qK\275@<6?\302\345\t?\3745=\275\366<6?\004\324\n?<\215L\275\nH6?\372\001\014?\2463_\275\263o6?{S\r?\351\245Z\2755\3735?.\210\022?\244\271&\275\305\2633?\016\305\375>\312\253\247\275H\3270?s\225\376>\201\010q\2753\016\023?\344\232\330>6\263\037>\250\366&?\356\316\350>\373!\371<PK#?4\275\351>\323^\013=\315\271\037?{\024\352>\023\370\'=\037\326\032?\300\353\350>\3038p=\303\022*?\367Q\347>\013\024\366<\342\021!?5\212\334>:\254\006=\256>%?\322\266\334>\021/\370<S\'\035?\177\

In [35]:
N_FILES = len(train_df)
CHUNK_SIZE = 512
N_PART = 1
FOLD = 4
part = 0

class CFG:
    seed = 42
    n_splits = 4

In [36]:
train_folds = train_df.copy()
train_folds['fold']=-1

num_bins = 5

# train_folds = train_folds.sample(frac=1, random_state=CFG.seed).reset_index(drop=True)
# gkfold = StratifiedGroupKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed) 
# print(f'{CFG.n_splits}fold training', len(train_folds), 'samples')
# for fold_idx, (train_idx, valid_idx) in enumerate(gkfold.split(train_folds, y=train_folds['sign'].values, groups=train_folds.participant_id)):
#     train_folds.loc[valid_idx,'fold'] = fold_idx
#     print(f'fold{fold_idx}:', 'train', len(train_idx), 'valid', len(valid_idx))
kfold = KFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed) 
print(f'{CFG.n_splits}fold training', len(train_folds), 'samples')
for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(train_folds)):
    train_folds.loc[valid_idx,'fold'] = fold_idx
    print(f'fold{fold_idx}:', 'train', len(train_idx), 'valid', len(valid_idx))
    
assert not (train_folds['fold']==-1).sum()
assert len(np.unique(train_folds['fold']))==CFG.n_splits
train_folds.head()

4fold training 67208 samples
fold0: train 50406 valid 16802
fold1: train 50406 valid 16802
fold2: train 50406 valid 16802
fold3: train 50406 valid 16802


,path,file_id,sequence_id,participant_id,phrase,fold
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse,1
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah,1
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier,3
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane,1
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road,0


In [37]:
%%time

import json
import os

DATASET_NAME = f'ISLFR-{CFG.n_splits}fold-randsplit'
### Create dataset Dataset if not exists 
# DATASET_NAME='nearestls'

!rm -rf /tmp/{DATASET_NAME}

os.makedirs(f'/tmp/{DATASET_NAME}', exist_ok=True)

with open('/dataset/input/d/ivanisaev/datasetapi/dataset.json') as f:
    dataset_creds = json.load(f)
    
os.environ['dataset_USERNAME'] = dataset_creds['username']
os.environ['dataset_KEY'] = dataset_creds['key']

!dataset datasets init -p /tmp/{DATASET_NAME}

with open(f'/tmp/{DATASET_NAME}/dataset-metadata.json') as f:
    dataset_meta = json.load(f)

dataset_meta['id'] = f'ivanisaev/{DATASET_NAME}'
dataset_meta['title'] = DATASET_NAME
with open(f'/tmp/{DATASET_NAME}/dataset-metadata.json', "w") as outfile:
    json.dump(dataset_meta, outfile)
print(dataset_meta)

!cp /tmp/{DATASET_NAME}/dataset-metadata.json /tmp/{DATASET_NAME}/meta.json
!ls /tmp/{DATASET_NAME}

!dataset datasets create -p /tmp/{DATASET_NAME} --public

Data package template written to: /tmp/ISLFR-4fold-randsplit/dataset-metadata.json
{'title': 'ISLFR-4fold-randsplit', 'id': 'ivanisaev/ISLFR-4fold-randsplit', 'licenses': [{'name': 'CC0-1.0'}]}
dataset-metadata.json  meta.json
Starting upload for file meta.json
100%|████████████████████████████████████████████| 110/110 [00:00<00:00, 118B/s]
Upload successful: meta.json (110B)
Dataset creation error: The requested title "ISLFR-4fold-randsplit" is already in use by a dataset. Please choose another title.
CPU times: user 104 ms, sys: 115 ms, total: 219 ms
Wall time: 8.25 s


In [38]:
!pwd /tmp

/dataset/working


In [39]:
# Put every image in a seperate TFRecord file
# Make Pairs of Views as input to the model


def split_dataframe(df, chunk_size = 10000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

# for fold in range(CFG.n_splits):

fold = 0

rows = train_folds[train_folds['fold']==fold]
chunks = split_dataframe(rows, CHUNK_SIZE)
part_size = len(chunks)//N_PART
last = (part+1)*part_size if part != N_PART - 1 else len(chunks)+1
chunks = chunks[part*part_size:last]

N = [len(x) for x in chunks]
_ = Parallel(n_jobs=cpu_count())(
    delayed(process_chunk)(x, f'/tmp/{DATASET_NAME}/fold{fold}-{i}-{n}.tfrecords')
    for i,(x,n) in enumerate(zip(chunks,N))
)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
23it [00:00, 228.72it/s]
46it [00:00, 213.35it/s]
68it [00:00, 213.95it/s]
92it [00:00, 220.47it/s]
115it [00:00, 216.30it/s]
137it [00:00, 213.43it/s]
159it [00:00, 198.98it/s]
180it [00:00, 185.62it/s]
199it [00:01, 175.15it/s]
217it [00:01, 167.77it/s]
248it [00:01, 188.85it/s]
1it [00:27, 27.18s/it]
0it [00:00, ?it/s]
14it [00:00, 134.81it/s]
28it [00:00, 137.62it/s]
42it [00:00, 137.98it/s]
59it [00:00, 146.92it/s]
74it [00:00, 147.14it/s]
89it [00:00, 132.83it/s]
103it [00:00, 133.58it/s]
119it [00:00, 139.85it/s]
137it [00:00, 151.59it/s]
153it [00:01, 152.02it/s]
170it [00:01, 154.70it/s]
196it [00:01, 148.57it/s]
1it [00:29, 29.25s/it]
0it [00:00, ?it/s]
17it [00:00, 166.74it/s]
34it [00:00, 162.81it/s]
51it [00:00, 155.78it/s]
67it [00:00, 147.30it/s]
83it [00:00, 144.96it/s]
98it [00:00, 141.31it/s]
113it [00:00, 127.22it/s]
126it [00:00, 116.23it/s]
139it [00:01, 119.20it/s]
153it [00:01, 123.91it/s]
166it [00:01, 117.41it/s]
180it [00:

KeyboardInterrupt: 

In [ ]:
%cd /tmp/ISLFR-4fold-randsplit

In [ ]:
!ls

In [ ]:
# print(tf. __version__) 

In [ ]:
from datetime import datetime
version_name = datetime.now().strftime("%Y%m%d-%H%M%S")
print(version_name)

In [ ]:
!dataset datasets version -m {version_name} -p /tmp/{DATASET_NAME} -r zip

In [ ]:
from dataset_datasets import datasetDatasets
print(datasetDatasets().get_gcs_path('islfr-4fold'))
print(datasetDatasets().get_gcs_path('islfr-4fold-randsplit'))
# print(datasetDatasets().get_gcs_path('islr-5fold-fold0'))
# print(datasetDatasets().get_gcs_path('islr-5fold-fold1'))
# print(datasetDatasets().get_gcs_path('islr-5fold-fold2'))
# print(datasetDatasets().get_gcs_path('islr-5fold-fold3'))
# print(datasetDatasets().get_gcs_path('islr-5fold-fold4'))

In [ ]:
# train = pd.read_csv('/dataset/input/asl-signs/train.csv')

In [ ]:
# train_size = 94477

In [ ]:
# train.shape

In [ ]:
# name = "/carpina/hope_&_faith/litle"
# print(name, '\n',label_to_num(name))

In [ ]:
# train_y = np.ones([train_size, max_str_len]) * -1
# train_label_len = np.zeros([train_size, 1])
# train_input_len = np.ones([train_size, 1]) * (num_of_timestamps-2)
# train_output = np.zeros([train_size])

# for i in range(train_size):
#     train_label_len[i] = len(train.loc[i, 'sign'])
#     train_y[i, 0:len(train.loc[i, 'sign'])]= label_to_num(train.loc[i, 'sign'])   

In [ ]:
# train_y[0]

In [ ]:
# tmp = train_y[0][1:3].copy()

In [ ]:
# tmp[0:2] = [5, 6]

In [ ]:
# tmp

In [ ]:
# train.loc[1, 'sign']

In [ ]:
# LAB_DICT['wait']